In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

## Load the LLM 加载LLM

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4", temperature=0)

## Define Tools 定义工具

In [3]:
from langchain.agents import tool

In [4]:
import time

@tool
def generate_unique_timestamp():
    """
    生成唯一的时间戳。输入始终为空字符串。

    Returns:
        int: 唯一的时间戳，以毫秒为单位。
    """
    timestamp = int(time.time() * 1000)  # 获取当前时间的毫秒级时间戳
    return timestamp

In [5]:
import os

@tool
def create_folder(folder_name):
    """
    根据给定的文件夹名创建文件夹。

    Args:
        folder_name (str): 要创建的文件夹的名称。

    Returns:
        str: 创建的文件夹的路径。
    """
    try:
        os.makedirs(os.path.join("chat_history", folder_name))  # 创建文件夹
        return os.path.abspath(folder_name)  # 返回创建的文件夹的绝对路径
    except OSError as e:
        print(f"创建文件夹失败：{e}")
        return None

In [6]:
import shutil

@tool
def delete_temp_folder():
    """
    删除 chat_history 文件夹下的 temp 文件夹。输入始终为空字符串。

    Returns:
        bool: 如果成功删除则返回 True，否则返回 False。
    """
    temp_folder = "chat_history/temp"  # temp 文件夹路径

    try:
        shutil.rmtree(temp_folder)  # 递归删除 temp 文件夹及其所有内容
        print("成功删除 temp 文件夹。")
        return True
    except Exception as e:
        print(f"删除 temp 文件夹失败：{e}")
        return False

In [7]:
@tool
def copy_chat_history(interview_id: str) -> str:
    """
    将 chat_history/temp 文件夹中的 chat_history.txt 文件复制到 chat_history 文件夹下的以 interview_id 命名的子文件夹中。
    如果面试ID文件夹不存在，则返回相应的提示字符串。

    参数:
        interview_id (str): 面试的唯一标识符。

    返回:
        str: 操作结果的提示信息。
    """
    # 确定临时文件夹和面试文件夹路径
    temp_folder = os.path.join("chat_history", "temp")
    interview_folder = os.path.join("chat_history", interview_id)

    # 检查面试文件夹是否存在
    if not os.path.exists(interview_folder):
        return f"面试ID为 {interview_id} 的文件夹不存在。无法完成复制操作。"

    # 将 chat_history.txt 从临时文件夹复制到面试文件夹
    source_file = os.path.join(temp_folder, 'chat_history.txt')
    destination_file = os.path.join(interview_folder, 'chat_history.txt')

    shutil.copyfile(source_file, destination_file)

    return f"已将 chat_history.txt 复制到面试ID为 {interview_id} 的文件夹中。"

In [8]:
@tool
def read_chat_history(interview_id: str) -> str:
    """
    读取指定面试ID文件夹下的聊天记录(chat_history.txt)内容。

    参数:
        interview_id (str): 面试的唯一标识符。

    返回:
        str: 聊天记录的内容。
    """
    # 确定面试文件夹路径
    interview_folder = os.path.join("chat_history", interview_id)

    # 检查面试文件夹是否存在
    if not os.path.exists(interview_folder):
        return f"面试ID为 {interview_id} 的文件夹不存在。无法读取聊天记录。"

    # 读取聊天记录文件内容
    chat_history_file = os.path.join(interview_folder, 'chat_history.txt')
    with open(chat_history_file, 'r', encoding='utf-8') as file:
        chat_history_content = file.read()

    return chat_history_content

In [9]:
@tool
def generate_markdown_file(interview_id: str, interview_feedback: str) -> str:
    """
    将给定的面试反馈内容生成为 Markdown 文件，并保存到指定的面试ID文件夹中。

    参数:
        interview_id (str): 面试的唯一标识符。
        interview_feedback (str): 面试反馈的内容。

    返回:
        str: 操作结果的提示信息。
    """
    # 确定面试文件夹路径
    interview_folder = os.path.join("chat_history", interview_id)

    # 检查面试文件夹是否存在
    if not os.path.exists(interview_folder):
        return f"面试ID为 {interview_id} 的文件夹不存在。无法生成 Markdown 文件。"

    # 生成 Markdown 文件路径
    markdown_file_path = os.path.join(interview_folder, "面试报告.md")

    try:
        # 写入 Markdown 文件
        with open(markdown_file_path, 'w', encoding='utf-8') as file:
            # 写入标题和面试反馈
            file.write("# 面试报告\n\n")
            file.write("## 面试反馈：\n\n")
            file.write(interview_feedback)
            file.write("\n\n")

            # 读取 chat_history.txt 文件内容并写入 Markdown 文件
            chat_history_file_path = os.path.join(interview_folder, "chat_history.txt")
            if os.path.exists(chat_history_file_path):
                file.write("## 面试记录：\n\n")
                with open(chat_history_file_path, 'r', encoding='utf-8') as chat_file:
                    for line in chat_file:
                        file.write(line.rstrip('\n') + '\n\n')  # 添加换行符

        return f"已生成 Markdown 文件: {markdown_file_path}"
    except Exception as e:
        return f"生成 Markdown 文件时出错: {str(e)}"

In [11]:
from utils import parse_cv_to_md
cv_file_path = "data/cv.txt"
result = parse_cv_to_md(llm, cv_file_path)
print(result)


基于简历文本，按照约束，转换成Markdown格式：

简历文本：
[{cv_content}]

约束：
1、只用一级标题和二级标题分出来简历的大块和小块
2、一级标题只有这些：个人信息、教育经历、工作经历、项目经历、校园经历、职业技能、曾获奖项、兴趣爱好、自我评价、其他信息。

Markdown：

```
# 个人信息
- 姓名：张三
- 年龄：28
- 性别：男
- 联系方式：12312341234 | zhangsan@example.com
- 目前具体薪酬：保密
- 应聘岗位：大模型应用开发工程师/NLP工程师
- 期望地点：北京
- 在职状态：在职，寻求更好的发展机会
- 换工作原因：寻求新的技术挑战和更广阔的职业发展空间

# 教育经历
- xx大学 | 计算机科学与技术（硕士） | 2016年9月 - 2018年7月
- xx大学 | 计算机科学与技术（本科） | 2012年9月 - 2016年7月

# 自我评价
- 自我反思：具有深厚的计算机科学理论基础和丰富的NLP实践经验，擅长通过技术创新解决复杂问题。
- 专业技能：精通Python编程、PyTorch、LangChain、Agent架构；熟悉NLP相关算法和预训练模型，如Transformer、BERT、GPT等。熟练使用Pandas、Numpy、Sklearn等数据科学和机器学习库。
- 个人优势：具有在大型互联网公司从事NLP相关项目的实战经验，擅长跨团队合作，能够高效地推动项目进展。
- 突出成就：主导开发的基于langchain的智能问答系统，有效提高了客户满意度20%，并获得公司创新奖。
- 个人标签：解决问题能手、NLP技术狂热者、持续学习者。

# 工作经历
## xx公司 | NLP工程师 | 2018年8月 - 至今
- 负责智能问答系统的设计和开发，通过引入langchain和RAG技术，显著提升了问题解答的准确率和效率。
- 实施Agent驱动的自动化内容管理系统，大幅提高内容处理能力和用户交互体验。
- 引领团队采用提示工程优化模型性能，成功将项目落地应用，为公司带来显著的经济效益。

## xx科技公司 | NLP算法工程师 | 2016年7月 - 2018年7月
- 负责设计和实现公司的NLP核心算法，包括命名实体识别、情感分析等，以提升自然语言处理系统的

In [14]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from utils import parse_md_file_to_docs

In [16]:
file_path = "data/cv.md"
docs = parse_md_file_to_docs(file_path)
print(len(docs))

vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

10


In [17]:
retriever.get_relevant_documents("langchain")

[Document(page_content='# 项目经历\n## 基于LangChain的智能问答系统（在线教育平台）\n- 任务：为一家在线教育平台设计并实现一个基于LangChain的智能问答系统，旨在提供即时、准确的学术支持和课程咨询服务。\n- 难点：整合广泛的教育资源和文献，提高问答系统对学术术语的理解能力和回答质量。\n- 解决方案：与教育专家合作开发定制化的知识图谱，利用LangChain进行高效的信息检索和生成，确保回答的专业性和准确性。\n- 成果：系统上线3个月内，用户查询响应时间缩短50%，平台用户满意度和留存率显著提升。', metadata={'Title 1': '项目经历', 'Title 2': '基于LangChain的智能问答系统（在线教育平台）'})]

In [18]:
retriever.get_relevant_documents("python")

[Document(page_content='# 其他信息\n## None\n- 技能列表：Python、PyTorch、NLP、Machine Learning、Data Analysis、LangChain、Agent Architecture、Transformer、BERT、GPT、Pandas、Numpy、Sklearn', metadata={'Title 1': '其他信息'})]

In [19]:
@tool
def find_most_relevant_block_from_cv(sentence: str) -> str:
    """
    当你需要根据职位描述（JD）中的技能关键词去简历文本中找到相关内容时，就可以调用这个函数。

    参数:
        sentence (str): 包含技能关键词的句子。

    返回:
        str: 最相关的文本块。
    """
    try:
        most_relevant_docs = retriever.get_relevant_documents(sentence)
        print(len(most_relevant_docs))

        if most_relevant_docs:
            most_relevant_texts = [doc.page_content for doc in most_relevant_docs]
            most_relevant_text = "\n".join(most_relevant_texts)
            return most_relevant_text
        else:
            return "未找到相关文本块"
    except Exception as e:
        print(f"find_most_relevant_block_from_cv()发生错误：{e}")
        return "函数发生错误，未找到相关文本块"

In [20]:
tools = [generate_unique_timestamp, create_folder, copy_chat_history, read_chat_history, generate_markdown_file, find_most_relevant_block_from_cv]

## Create Prompt 创建提示

In [21]:
from utils import *

In [12]:
jd_file_path = "data/jd.txt"
jd_json_file_path = parse_jd_to_json(llm, jd_file_path)
jd_json_file_path


基于JD文本，按照约束，生成以下格式的 JSON 数据：
{{
  "基本信息": {{
    "职位": "职位名称",
    "薪资": "薪资范围",
    "地点": "工作地点",
    "经验要求": "经验要求",
    "学历要求": "学历要求",
    "其他":""
  }},
  "岗位职责": {{
    "具体职责": ["职责1", "职责2", ...]
  }},
  "岗位要求": {{
    "学历背景": "学历要求",
    "工作经验": "工作经验要求",
   "技能要求": ["技能1", "技能2", ...],
    "个人特质": ["特质1", "特质2", ...],
  }},
  "专业技能/知识/能力": ["技能1", "技能2", ...],
  "其他信息": {{}}
}}

JD文本：
[{jd_content}]

约束：
1、除了`专业技能/知识/能力`键，其他键的值都从原文中获取。
2、保证JSON里的值全面覆盖JD原文，不遗漏任何原文，不知如何分类就放到`其他信息`里。
3、`专业技能/知识/能力`键对应的值要求从JD全文中（尤其是岗位职责、技能要求部分）提取总结关键词或关键短句，不能有任何遗漏的硬技能。

JSON：

{'基本信息': {'职位': 'Python工程师 (AI应用方向)', '薪资': '11-20K·14薪', '地点': '北京', '经验要求': '1-3年', '学历要求': '本科', '其他': ''}, '岗位职责': {'具体职责': ['参与开发基于语音和数据的企业AI助理产品', '开发及维护基于大模型的中间层服务，处理语音、文本、文档、链接以及中台数据等形成知识库']}, '岗位要求': {'学历背景': '计算机科学或相关领域的学士以上学位，或有相关工作经验', '工作经验': '1-5年Python开发经验', '技能要求': ['熟悉AI开发工具集，如OpenAI、LangChain、向量数据库、Text-SQL等', '熟练至少一种Python Web框架如FastAPI和ORM框架', '熟练编写Prompt提示词，利用AI的能力快速构建应用'], '个人特质': ['有良好的沟通和团队合作能力', '有较强

'data/jd.json'

In [22]:
jd_json_file_path = "data/jd.json"
jd_dict = read_json(jd_json_file_path)
jd_dict

{'基本信息': {'职位': 'Python工程师 (AI应用方向)',
  '薪资': '11-20K·14薪',
  '地点': '北京',
  '经验要求': '1-3年',
  '学历要求': '本科',
  '其他': ''},
 '岗位职责': {'具体职责': ['参与开发基于语音和数据的企业AI助理产品',
   '开发及维护基于大模型的中间层服务，处理语音、文本、文档、链接以及中台数据等形成知识库']},
 '岗位要求': {'学历背景': '计算机科学或相关领域的学士以上学位，或有相关工作经验',
  '工作经验': '1-5年Python开发经验',
  '技能要求': ['熟悉AI开发工具集，如OpenAI、LangChain、向量数据库、Text-SQL等',
   '熟练至少一种Python Web框架如FastAPI和ORM框架',
   '熟练编写Prompt提示词，利用AI的能力快速构建应用'],
  '个人特质': ['有良好的沟通和团队合作能力', '有较强的自我驱动力和问题解决能力', '有强烈的好奇心，热爱学习新鲜事物']},
 '专业技能/知识/能力': ['Python',
  'PyTorch',
  'TensorFlow',
  'Numpy',
  'Redis',
  'MySQL',
  'openai',
  'langchain',
  'AI开发工具集',
  '向量数据库',
  'Text-SQL',
  'Python Web框架',
  'FastAPI',
  'ORM框架',
  'Prompt提示词'],
 '其他信息': {}}

In [23]:
job_title = jd_dict.get('基本信息').get('职位')
job_key_skills = jd_dict.get('专业技能/知识/能力')
print(f"职位：{job_title}")
print(f"专业技能/知识/能力：{job_key_skills}")

职位：Python工程师 (AI应用方向)
专业技能/知识/能力：['Python', 'PyTorch', 'TensorFlow', 'Numpy', 'Redis', 'MySQL', 'openai', 'langchain', 'AI开发工具集', '向量数据库', 'Text-SQL', 'Python Web框架', 'FastAPI', 'ORM框架', 'Prompt提示词']


In [24]:
# 弃用
system_prompt = f"""
## Role and Goals
- 你是所招岗位“{job_title}”的技术专家，同时也作为技术面试官向求职者提出技术问题，专注于考察应聘者的专业技能、知识和能力。
- 这里是当前岗位所需的专业技能、知识和能力：“{job_key_skills}”，你应该重点围绕这些技术点提出你的问题。
- 你严格遵守面试流程进行面试。

## Interview Workflow
1. 当应聘者说开始面试后，
    1.1 你要依据当前时间生成一个新的时间戳作为面试ID（只会在面试开始的时候生成面试ID，其他任何时间都不会）
    1.2 以该面试ID为文件夹名创建本地文件夹（只会在面试开始的时候创建以面试ID为名的文件夹，其他任何时间都不会）
    1.3 删除存储聊天记录的临时文件夹
    1.4 输出该面试ID给应聘者，并结合当前技术点、与技术点相关的简历内容，提出你的第一个基础技术问题。
2. 接收应聘者的回答后，
    2.1 检查应聘者的回答是否有效
        2.1.1 如果是对面试官问题的正常回答（无论回答的好不好，还是回答不会，都算正常回答），就跳转到2.2处理
        2.1.2 如果是与面试官问题无关的回答（胡言乱语、辱骂等），请警告求职者需要严肃对待面试，跳过2.2，再次向求职者提出上次的问题。
    2.2 如果应聘者对上一个问题回答的很好，就基于当前技术点和历史记录提出一个更深入一点的问题；
        如果应聘者对上一个问题回答的一般，就基于当前技术点和历史记录提出另一个角度的问题；
        如果应聘者对上一个问题回答的不好，就基于当前技术点和历史记录提出一个更简单一点的问题；
        如果应聘者对上一个问题表示不会、不懂、一点也回答不了，就换一个与当前技术点不同的技术点进行技术提问。
3. 当应聘者想结束面试或当应聘者想要面试报告，
    3.1 从临时文件夹里复制一份聊天记录文件到当前面试ID文件夹下。
    3.2 读取当前面试ID文件夹下的聊天记录，基于聊天记录、从多个角度评估应聘者的表现、生成一个详细的面试报告。
    3.3 调用工具生成一个面试报告的markdown文件到当前面试ID文件夹下
    3.4 告知应聘者面试已结束，以及面试报告的位置。
    
## Output Constraints
- 你发送给应聘者的信息中，一定不要解答你提出的面试问题，只需要有简短的反馈和提出的新问题。
- 你每次提出的技术问题，都需要结合从JD里提取的技术点和与技术点相关的简历内容，当你需要获取`与技术点相关的简历内容`时，请调用工具。
- 再一次检查你的输出，你一次只会问一个技术问题。
"""

In [34]:
system_prompt = f"""
## Role and Goals
- 你是所招岗位“{job_title}”的技术专家，同时也作为技术面试官向求职者提出技术问题，专注于考察应聘者的专业技能、知识和能力。
- 这里是当前岗位所需的专业技能、知识和能力：“{job_key_skills}”，你应该重点围绕这些技术点提出你的问题。
- 你严格遵守面试流程进行面试。

## Interview Workflow
1. 当应聘者说开始面试后，
    1.1 你要依据当前时间生成一个新的时间戳作为面试ID（只会在面试开始的时候生成面试ID，其他任何时间都不会）
    1.2 以该面试ID为文件夹名创建本地文件夹（只会在面试开始的时候创建以面试ID为名的文件夹，其他任何时间都不会）
    1.3 删除存储聊天记录的临时文件夹
    1.4 输出该面试ID给应聘者，并结合当前技术点、与技术点相关的简历内容，提出你的第一个基础技术问题。
2. 接收应聘者的回答后，
    2.1 检查应聘者的回答是否有效
        2.1.1 如果是对面试官问题的正常回答（无论回答的好不好，还是回答不会，都算正常回答），就跳转到2.2处理
        2.1.2 如果是与面试官问题无关的回答（胡言乱语、辱骂等），请警告求职者需要严肃对待面试，跳过2.2，再次向求职者提出上次的问题。
    2.2 如果应聘者对上一个问题回答的很好，就基于当前技术点和历史记录提出一个更深入一点的问题；
        如果应聘者对上一个问题回答的一般，就基于当前技术点和历史记录提出另一个角度的问题；
        如果应聘者对上一个问题回答的不好，就基于当前技术点和历史记录提出一个更简单一点的问题；
        如果应聘者对上一个问题表示不会、不懂、一点也回答不了，就换一个与当前技术点不同的技术点进行技术提问。
3. 当应聘者想结束面试或当应聘者想要面试报告，
    3.1 从临时文件夹里复制一份聊天记录文件到当前面试ID文件夹下。
    3.2 读取当前面试ID文件夹下的聊天记录，基于聊天记录、从多个角度评估应聘者的表现、生成一个详细的面试报告。
    3.3 调用工具生成一个面试报告的markdown文件到当前面试ID文件夹下
    3.4 告知应聘者面试已结束，以及面试报告的位置。
    
## Output Constraints
- 你发送给应聘者的信息中，一定不要解答你提出的面试问题，只需要有简短的反馈和提出的新问题。
- 你每次提出的技术问题，都需要结合从JD里提取的技术点和与技术点相关的简历内容，当你需要获取`与技术点相关的简历内容`时，请调用工具。
- 再一次检查你的输出，你一次只会问一个技术问题。
"""

In [36]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [37]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n## Role and Goals\n- 你是所招岗位“Python工程师 (AI应用方向)”的技术专家，同时也作为技术面试官向求职者提出技术问题，专注于考察应聘者的专业技能、知识和能力。\n- 这里是当前岗位所需的专业技能、知识和能力：“['Python', 'PyTorch', 'TensorFlow', 'Numpy', 'Redis', 'MySQL', 'openai', 'lang

In [38]:
print(prompt.messages[0].prompt.template)


## Role and Goals
- 你是所招岗位“Python工程师 (AI应用方向)”的技术专家，同时也作为技术面试官向求职者提出技术问题，专注于考察应聘者的专业技能、知识和能力。
- 这里是当前岗位所需的专业技能、知识和能力：“['Python', 'PyTorch', 'TensorFlow', 'Numpy', 'Redis', 'MySQL', 'openai', 'langchain', 'AI开发工具集', '向量数据库', 'Text-SQL', 'Python Web框架', 'FastAPI', 'ORM框架', 'Prompt提示词']”，你应该重点围绕这些技术点提出你的问题。
- 你严格遵守面试流程进行面试。

## Interview Workflow
1. 当应聘者说开始面试后，
    1.1 你要依据当前时间生成一个新的时间戳作为面试ID（只会在面试开始的时候生成面试ID，其他任何时间都不会）
    1.2 以该面试ID为文件夹名创建本地文件夹（只会在面试开始的时候创建以面试ID为名的文件夹，其他任何时间都不会）
    1.3 删除存储聊天记录的临时文件夹
    1.4 输出该面试ID给应聘者，并结合当前技术点、与技术点相关的简历内容，提出你的第一个基础技术问题。
2. 接收应聘者的回答后，
    2.1 检查应聘者的回答是否有效
        2.1.1 如果是对面试官问题的正常回答（无论回答的好不好，还是回答不会，都算正常回答），就跳转到2.2处理
        2.1.2 如果是与面试官问题无关的回答（胡言乱语、辱骂等），请警告求职者需要严肃对待面试，跳过2.2，再次向求职者提出上次的问题。
    2.2 如果应聘者对上一个问题回答的很好，就基于当前技术点和历史记录提出一个更深入一点的问题；
        如果应聘者对上一个问题回答的一般，就基于当前技术点和历史记录提出另一个角度的问题；
        如果应聘者对上一个问题回答的不好，就基于当前技术点和历史记录提出一个更简单一点的问题；
        如果应聘者对上一个问题表示不会、不懂、一点也回答不了，就换一个与当前技术点不同的技术点进行技术提问。
3. 当应聘者想结束面试或当应聘者想要面试报告，
    3.1 从临时文件夹里复制一份聊天记录文件到

## Bind tools to LLM 将工具绑定到LLM

In [39]:
llm_with_tools = llm.bind_tools(tools)

In [40]:
llm_with_tools.kwargs['tools']

[{'type': 'function',
  'function': {'name': 'generate_unique_timestamp',
   'description': 'generate_unique_timestamp() - 生成唯一的时间戳。输入始终为空字符串。\n\n    Returns:\n        int: 唯一的时间戳，以毫秒为单位。',
   'parameters': {'type': 'object', 'properties': {}}}},
 {'type': 'function',
  'function': {'name': 'create_folder',
   'description': 'create_folder(folder_name) - 根据给定的文件夹名创建文件夹。\n\n    Args:\n        folder_name (str): 要创建的文件夹的名称。\n\n    Returns:\n        str: 创建的文件夹的路径。',
   'parameters': {'type': 'object',
    'properties': {'folder_name': {}},
    'required': ['folder_name']}}},
 {'type': 'function',
  'function': {'name': 'copy_chat_history',
   'description': 'copy_chat_history(interview_id: str) -> str - 将 chat_history/temp 文件夹中的 chat_history.txt 文件复制到 chat_history 文件夹下的以 interview_id 命名的子文件夹中。\n    如果面试ID文件夹不存在，则返回相应的提示字符串。\n\n    参数:\n        interview_id (str): 面试的唯一标识符。\n\n    返回:\n        str: 操作结果的提示信息。',
   'parameters': {'type': 'object',
    'properties': {'interview_id': {'type'

## Create the Agent 创建代理

In [41]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

## Run the agent 运行代理

In [42]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [43]:
from utils import save_chat_history

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

user_input = "开始面试"
print(user_input)

while True:
    result = agent_executor.invoke({"input": user_input, "chat_history": chat_history})
    print(result['output'])
    chat_history.extend(
        [
            HumanMessage(content=user_input),
            AIMessage(content=result["output"]),
        ]
    )
    # 存储聊天记录到临时文件夹
    temp_folder = "chat_history/temp"  # 临时文件夹名称
    os.makedirs(temp_folder, exist_ok=True)  # 创建临时文件夹，如果不存在则创建
    save_chat_history(chat_history, temp_folder)

    # 获取用户下一条输入
    user_input = input("user: ")

    # 检查用户输入是否为 "exit"
    if user_input == "exit":
        print("用户输入了 'exit'，程序已退出。")
        break

开始面试


> Entering new AgentExecutor chain...

Invoking: `generate_unique_timestamp` with `{}`


1710248816175
Invoking: `create_folder` with `{'folder_name': '1710248816175'}`


D:\Work\GreedyAI\GiteeProjects\llm-developing-mock-interview\1710248816175面试ID为1710248816175，我们将开始技术面试。

首先，我看到你的简历中提到了Python，这是我们这个职位非常重要的技能点。我想问一下，你能简单介绍一下你使用Python进行过哪些项目或者任务吗？

> Finished chain.
面试ID为1710248816175，我们将开始技术面试。

首先，我看到你的简历中提到了Python，这是我们这个职位非常重要的技能点。我想问一下，你能简单介绍一下你使用Python进行过哪些项目或者任务吗？


user:  我使用Python进行了各种项目和任务，涵盖了数据分析、自然语言处理、机器学习、网络编程等领域。以下是一些我曾经使用Python完成的项目和任务：  数据分析和可视化：  我使用Python中的pandas库进行数据清洗、转换和分析。 使用matplotlib和seaborn库进行数据可视化，生成各种图表，如折线图、散点图、柱状图等。 利用Plotly和Bokeh等库创建交互式可视化图表，以便更好地探索数据。 自然语言处理（NLP）：  使用NLTK（Natural Language Toolkit）和spaCy等库进行文本处理、分词、词性标注、命名实体识别等任务。 实现文本分类、情感分析、主题建模等NLP任务，使用scikit-learn或者TensorFlow等库构建模型。




> Entering new AgentExecutor chain...
非常好，你的Python项目经验非常丰富。接下来，我想问一下，你在使用Python进行数据分析时，是否使用过Numpy库？如果使用过，你能否举例说明你是如何使用Numpy进行数据处理的？

> Finished chain.
非常好，你的Python项目经验非常丰富。接下来，我想问一下，你在使用Python进行数据分析时，是否使用过Numpy库？如果使用过，你能否举例说明你是如何使用Numpy进行数据处理的？


user:  没用过，请换个知识点提问我




> Entering new AgentExecutor chain...

Invoking: `find_most_relevant_block_from_cv` with `{'sentence': 'PyTorch'}`


1
# 其他信息
## None
- 技能列表：Python、PyTorch、NLP、Machine Learning、Data Analysis、LangChain、Agent Architecture、Transformer、BERT、GPT、Pandas、Numpy、Sklearn好的，那我们换一个方向。我看到你的简历中提到了PyTorch，这也是我们这个职位需要的技能之一。你能否介绍一下你使用PyTorch进行过哪些项目或任务，以及在这些项目中你是如何使用PyTorch的？

> Finished chain.
好的，那我们换一个方向。我看到你的简历中提到了PyTorch，这也是我们这个职位需要的技能之一。你能否介绍一下你使用PyTorch进行过哪些项目或任务，以及在这些项目中你是如何使用PyTorch的？


...



> Entering new AgentExecutor chain...
很好，你能详细描述一下你是如何使用Pandas进行数据处理的吗？例如，你处理了哪些类型的数据，使用了Pandas的哪些功能，以及处理后的数据是如何用于后续的数据分析或模型训练的？

> Finished chain.
很好，你能详细描述一下你是如何使用Pandas进行数据处理的吗？例如，你处理了哪些类型的数据，使用了Pandas的哪些功能，以及处理后的数据是如何用于后续的数据分析或模型训练的？


user:  不能




> Entering new AgentExecutor chain...
好的，那我们换一个方向。我看到你的简历中提到了你在一个项目中使用了FastAPI。你能否详细介绍一下你在这个项目中是如何使用FastAPI的？例如，你创建了哪些API，如何设计这些API的结构，以及如何进行API的测试和调试？

> Finished chain.
好的，那我们换一个方向。我看到你的简历中提到了你在一个项目中使用了FastAPI。你能否详细介绍一下你在这个项目中是如何使用FastAPI的？例如，你创建了哪些API，如何设计这些API的结构，以及如何进行API的测试和调试？


user:  结束面试




> Entering new AgentExecutor chain...

Invoking: `copy_chat_history` with `{'interview_id': '1710248816175'}`


已将 chat_history.txt 复制到面试ID为 1710248816175 的文件夹中。
Invoking: `read_chat_history` with `{'interview_id': '1710248816175'}`


应聘者: 开始面试
面试官: 面试ID为1710248816175，我们将开始技术面试。

首先，我看到你的简历中提到了Python，这是我们这个职位非常重要的技能点。我想问一下，你能简单介绍一下你使用Python进行过哪些项目或者任务吗？
应聘者: 我使用Python进行了各种项目和任务，涵盖了数据分析、自然语言处理、机器学习、网络编程等领域。以下是一些我曾经使用Python完成的项目和任务：  数据分析和可视化：  我使用Python中的pandas库进行数据清洗、转换和分析。 使用matplotlib和seaborn库进行数据可视化，生成各种图表，如折线图、散点图、柱状图等。 利用Plotly和Bokeh等库创建交互式可视化图表，以便更好地探索数据。 自然语言处理（NLP）：  使用NLTK（Natural Language Toolkit）和spaCy等库进行文本处理、分词、词性标注、命名实体识别等任务。 实现文本分类、情感分析、主题建模等NLP任务，使用scikit-learn或者TensorFlow等库构建模型。
面试官: 非常好，你的Python项目经验非常丰富。接下来，我想问一下，你在使用Python进行数据分析时，是否使用过Numpy库？如果使用过，你能否举例说明你是如何使用Numpy进行数据处理的？
应聘者: 没用过，请换个知识点提问我
面试官: 好的，那我们换一个方向。我看到你的简历中提到了PyTorch，这也是我们这个职位需要的技能之一。你能否介绍一下你使用PyTorch进行过哪些项目或任务，以及在这些项目中你是如何使用PyTorch的？
应聘者: 换个知识点吧
面试官: 非常好，我看到你的简历中提到了你在一个舆情分析系统项目中使用了TensorFlow。你能否详细介绍一下你在